In [286]:
import pandas as pd
import boto3
import botocore
from io import StringIO

In [287]:
# AWS credentials
# Download the credentials in teams and save in your local machine, update the path as needed
file_path = 'C:/churn/bucketcredentials.csv'
credentials_df = pd.read_csv(file_path)

aws_access_key_id = credentials_df['aws_access_key_id'].iloc[0]
aws_secret_access_key = credentials_df['aws_secret_access_key'].iloc[0]
bucket_name = credentials_df['bucket_name'].iloc[0]


In [288]:
# Create an S3 client
s3_client = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

In [289]:
# Set param
# for testing purposes you can change the path so you will not overwrite others output file
env = "test" #  dev, test, staging, prod 
model = "model1" # model1, model2, model3
mainsource = "SourceDataSet/bank_data_train.csv"
envraw = f'{env}/raw/bank_data_train.csv'

#for pre processing and feature engineering we only have one source 
sourcedata = "raw" 
targetdata = "processed"
sourcefoldername = f'{env}/{sourcedata}'
targetfoldername = f'{env}/{targetdata}'
sourcefilename = "bank_data_train.csv"
targetfilename = "bank_data_test.csv"

# for modeling may vary on environment, and user
# sourcedata = "processed" 
# targetdata = "final"
# sourcefoldername = f'{env}/{sourcedata}'
# targetfoldername = f'{env}/{targetdata}/{model}'
# sourcefilename = "bank_data_test.csv"
# targetfilename = "bank_data.csv"



In [290]:

def check_and_transfer_file(bucket_name, source_key, raw_destination_key):
    try:
        # Check if the file exists in the destination location
        s3_client.head_object(Bucket=bucket_name, Key=raw_destination_key)
    except botocore.exceptions.ClientError as e:
        print('test')
        if e.response['Error']['Code'] == "404":
            s3_client.copy_object(Bucket=bucket_name, CopySource=f"{bucket_name}/{source_key}", Key=raw_destination_key)
        else:
            print("An error occurred:", e)
# 

def check_file_exists(bucket_name, file_key):
    try:
        # Check if the file exists
        s3_client.head_object(Bucket=bucket_name, Key=file_key)
        return True
        #print(f"File '{file_key}' exists.")
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            return False
        else:
            return False
            


In [291]:
# SourceDataSet in env
check_and_transfer_file(bucket_name, mainsource, envraw)

In [292]:
# Read the CSV file from S3 into a DataFrame
input_file_key = f'{sourcefoldername}/{sourcefilename}'
output_file_key = f'{targetfoldername}/{targetfilename}'


In [293]:
# Check if raw data exists
file_exists = check_file_exists(bucket_name,input_file_key)
if file_exists:
    obj = s3_client.get_object(Bucket=bucket_name, Key=input_file_key)
    df = pd.read_csv(obj['Body'])

In [294]:
# Extract the first two rows of the DataFrame
#df_head = df.head(1)

In [295]:
# Convert the DataFrame to a CSV string
csv_buffer = StringIO()
df_head.to_csv(csv_buffer, index=False)


In [296]:

# Upload the new CSV to the desired folder in the S3 bucket
s3_client.put_object(Bucket=bucket_name, Key=output_file_key, Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'XM39CTEMNSTS75BQ',
  'HostId': 'BNN+HlCh8KuhPrk3BEoN7gyyxdohGUGYseCsUff5pkPV7DAf+AZW2GR5Nil9JAJNSrjfkLeDMpA=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'BNN+HlCh8KuhPrk3BEoN7gyyxdohGUGYseCsUff5pkPV7DAf+AZW2GR5Nil9JAJNSrjfkLeDMpA=',
   'x-amz-request-id': 'XM39CTEMNSTS75BQ',
   'date': 'Tue, 19 Mar 2024 23:32:58 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"c3ece34322f24c713d7483a700cb53fd"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"c3ece34322f24c713d7483a700cb53fd"',
 'ServerSideEncryption': 'AES256'}